# Scoring your trained model

In the cell below, please load your model into `model`. Also if you used an image size for your input images that *isn't* 224x224, you'll need to set `image_size` to the size you used. The scoring code assumes square input images.

For example, this is how I loaded in my checkpoint:

```python
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
    
    def __init__(self, in_features, hidden_features, 
                       out_features, drop_prob=0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p=drop_prob)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(25088, checkpoint['hidden'], 102)
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('/home/workspace/classifier.pt')
```

Your exact code here will depend on how you defined your network in the project. Make sure you use the absolute path to your checkpoint which should have been uploaded to the `/home/workspace` directory.

Run the cell, then after loading the data, press "Test Code" below. This can take a few minutes or more depending on the size of your network. Your model needs  to reach **at least 20% accuracy** on the test set to be recorded.

In [ ]:
!pip install gdown==3.6.0

In [ ]:
my_file_id = ""

In [ ]:
!gdown https://drive.google.com/uc?id={my_file_id}

In [ ]:
# VGG
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models
from collections import OrderedDict

def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path,map_location='cpu')

    model = models.vgg19(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = nn.Sequential(OrderedDict([
   ('fc1', nn.Linear(25088, 4096)),
   ('relu', nn.ReLU()),
   ('fc2', nn.Linear(4096, 102)),
   ('output', nn.LogSoftmax(dim=1))
]))

    model.load_state_dict(checkpoint['state_dict'],strict=False)

    return model

# Load your model to this variable
model = load_checkpoint('/home/workspace/classifier.pth')
model.eval()
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

In [1]:
# FOR DENSE NET
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
  def __init__(self,n_hidden1=1024,n_hidden2=512):
    super().__init__()
    
    self.fc1=nn.Linear(2208,n_hidden1)
    self.relu1=nn.ReLU()
    self.fc2=nn.Linear(n_hidden1,n_hidden2)
    self.relu2=nn.ReLU()
    self.output=nn.Linear(n_hidden2,102)
    self.dropout=nn.Dropout(p=0.25)
    
  def forward(self,x):
    
    x=self.relu1(self.fc1(x))
    x=self.dropout(x)
    x=self.relu2(self.fc2(x))
    x=self.dropout(x)
    x=self.output(x)
    
    return x


def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path,map_location='cpu')

    model = models.densenet161(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier()

    model.load_state_dict(checkpoint['state'],strict=False)

    return model

# Load your model to this variable
model = load_checkpoint('/home/workspace/net14_suite_0.01LR-SCHPLAT_sgd_densenet161_brightness_unfrozen_0.999.pt')
model.eval()
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.torch/models/densenet161-8d451a50.pth
100%|██████████| 115730790/115730790 [00:04<00:00, 24466520.52it/s]
